# Python收发邮件
- 使用框架自带的email模块
- 参考文档：
    - 官方文档：https://docs.python.org/3/library/email.html#module-email
    - email源码：https://github.com/python/cpython/tree/3.7/Lib/email
    - multipart使用http方式传文件http://www.cnblogs.com/yydcdut/p/3736667.html
    - multipart MIME rfc1341 https://www.w3.org/Protocols/rfc1341/7_2_Multipart.html
名词解释：
- 邮件：RFC822规定的
- RFC822：https://www.ietf.org/rfc/rfc822.txt
- RFC822解读：http://blog.51cto.com/003317/611104
- Multipart MIME：MIME类型 https://developer.mozilla.org/zh-CN/docs/Web/HTTP/Basics_of_HTTP/MIME_types

## 工作流程
### 收邮件
- 连接服务器，并进行认证
- 通过list方法，返回resp，mails列表，octets，其中mails是邮件列表（编号，字节数），元素形如：b'1 18809'
- 通过server.retr()方法获取邮件，返回resp, lines列表, octets，其中lines列表是个字节流列表，元素形如：b'Date: Wed, 25 Jul 2018 16:51:42 +0800'
- 将获取到的lines字节流列表转换成字符串列表
- 将字符串列表通过Parser().parsestr()方法解析成Message对象
- 调用Message对象的的get()方法获取指定信息，调用walk方法遍历Message

常用api
```
get(name, failobj=None)
Return the value of the named header field. This is identical to __getitem__() except that optional failobj is returned if the named header is missing (failobj defaults to None).

walk()
The walk() method is an all-purpose generator which can be used to iterate over all the parts and subparts of a message object tree, in depth-first traversal order. You will typically use walk() as the iterator in a for loop; each iteration returns the next subpart.

Here’s an example that prints the MIME type of every part of a multipart message structure:

>>> for part in msg.walk():
...     print(part.get_content_type())
multipart/report
text/plain
message/delivery-status
text/plain
text/plain
message/rfc822
text/plain
walk iterates over the subparts of any part where is_multipart() returns True, even though msg.get_content_maintype() == 'multipart' may return False. We can see this in our example by making use of the _structure debug helper function:

>>> for part in msg.walk():
...     print(part.get_content_maintype() == 'multipart',
...           part.is_multipart())
True True
False False
False True
False False
False False
False True
False False
>>> _structure(msg)
multipart/report
    text/plain
    message/delivery-status
        text/plain
        text/plain
    message/rfc822
        text/plain
Here the message parts are not multiparts, but they do contain subparts. is_multipart() returns True and walk descends into the subparts.
```

### 发邮件
- 参考文档：https://github.com/python/cpython/blob/3.7/Lib/smtplib.py
- 发送普通文本文件
- 发送带附件的邮件
- 发送一个文件夹
- 发送HTML邮件

- 1. 连接邮件服务器
- 2. 登录login
- 3. 构造邮件
- 4. 发送

因为还没用到，先不写




In [25]:
import poplib
import datetime
import time
from email.parser import Parser
from email.header import decode_header
from email.utils import parseaddr 


host = 'xxx'
username = 'xxx'
password = 'xxx'

#连接邮件服务器
server = poplib.POP3(host)
server.user(username)
server.pass_(password)
print(server)


gaierror: [Errno 8] nodename nor servname provided, or not known

In [2]:

#返回tuple，其中mails是个列表，列表元素是msg编号(从1开始计数)，字节数
resp, mails, octets = server.list()
print(mails)
print(mails[1].decode())

[b'1 18809', b'2 553736', b'3 26572', b'4 24471', b'5 19951', b'6 18600', b'7 19394', b'8 29771', b'9 19429', b'10 1135361', b'11 1439110', b'12 4366696', b'13 50139', b'14 320362', b'15 501398', b'16 11308', b'17 11203', b'18 14216', b'19 8448', b'20 17981', b'21 3085882', b'22 3623115', b'23 10291', b'24 274877', b'25 16621', b'26 10291', b'27 62649228', b'28 2685522', b'29 8208']
2 553736


In [26]:
#Retrieve whole message number which, and set its seen flag. 
#Result is in form (response, ['line', ...], octets).
#抽取最后一封邮件的信息的lines
resp, lines, octets = server.retr(len(mails))
print(resp)
print(type(lines[0]))
# lines就是邮件主体,是一个字节流列表
#print(lines)
print(octets)

ConnectionResetError: [Errno 54] Connection reset by peer

In [27]:
# 将字节流列表转换成字符串列表
msg_content = b'\r\n'.join(lines).decode()

#print(msg_content)

In [28]:
#Parser可以吧一个字符串解析成一个email.Message对象
msg = Parser().parsestr(msg_content)
#print(type(msg))
# 返回一个生成器对象
#print('walk:',msg.walk())
#for part in msg.walk():
#    print('part:',part)
#print(msg.get('Subject'))
#print(msg.get('From'))
#print(msg)

In [ ]:
import smtplib

# Import the email modules we'll need
from email.message import EmailMessage

'''
# Open the plain text file whose name is in textfile for reading.
with open(textfile) as fp:
    # Create a text/plain message
    msg = EmailMessage()
    msg.set_content(fp.read())
'''
textfile = 'hello'
# me == the sender's email address
# you == the recipient's email address
msg['Subject'] = 'The contents of %s' % textfile
msg['From'] = me
msg['To'] = you

# Send the message via our own SMTP server.
s = smtplib.SMTP('localhost')
s.send_message(msg)
s.quit()

In [ ]:
# Import smtplib for the actual sending function
import smtplib

# And imghdr to find the types of our images
import imghdr

# Here are the email package modules we'll need
from email.message import EmailMessage

# Create the container email message.
msg = EmailMessage()
msg['Subject'] = 'Our family reunion'
# me == the sender's email address
# family = the list of all recipients' email addresses
msg['From'] = me
msg['To'] = ', '.join(family)
msg.preamble = 'Our family reunion'

# Open the files in binary mode.  Use imghdr to figure out the
# MIME subtype for each specific image.
for file in pngfiles:
    with open(file, 'rb') as fp:
        img_data = fp.read()
    msg.add_attachment(img_data, maintype='image',
                                 subtype=imghdr.what(None, img_data))

# Send the email via our own SMTP server.
with smtplib.SMTP('localhost') as s:
    s.send_message(msg)